In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow 
from tensorflow.keras.datasets import cifar10

In [ ]:
(X_train,y_train), (X_test,y_test) = cifar10.load_data()

In [ ]:
X_test = X_test/255
X_train = X_train/255

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)
y_train[0]

In [ ]:
X_train = X_train.reshape(X_train.shape[0],32,32,3)
X_test = X_test.reshape(X_test.shape[0],32,32,3)

In [ ]:
input_img = tensorflow.keras.layers.Input((32, 32, 3))

encoded = tensorflow.keras.layers.Conv2D(64, (4,4), padding='same', name='e1')(input_img)
encoded = tensorflow.keras.layers.Activation('relu', name = 'e2')(encoded)
encoded = tensorflow.keras.layers.MaxPool2D(name = 'e3')(encoded)

encoded = tensorflow.keras.layers.Conv2D(32, (4,4), padding='same', name='e4')(encoded)
encoded = tensorflow.keras.layers.Activation('relu', name = 'e5')(encoded)
encoded = tensorflow.keras.layers.MaxPool2D(name = 'e6')(encoded)

encoded = tensorflow.keras.layers.BatchNormalization(name = 'e7')(encoded)

encoded = tensorflow.keras.layers.Conv2D(16, (4,4), padding='same', name='e8')(encoded)
encoded = tensorflow.keras.layers.Activation('relu', name = 'e9')(encoded)


decoded = tensorflow.keras.layers.Conv2D(16, (4,4), padding='same', name='d1')(encoded)
decoded = tensorflow.keras.layers.Activation('relu', name='d2')(decoded)
decoded = tensorflow.keras.layers.UpSampling2D(name = 'd3')(decoded)

decoded = tensorflow.keras.layers.BatchNormalization(name = 'd4')(decoded)

decoded = tensorflow.keras.layers.Conv2D(32, (4,4), padding='same', name='d5')(decoded)
decoded = tensorflow.keras.layers.Activation('relu', name='d6')(decoded)
decoded = tensorflow.keras.layers.UpSampling2D(name ='d7')(decoded)

decoded = tensorflow.keras.layers.Conv2D(64, (4,4), padding='same', name='d8')(decoded)

# Renormalize the image to (32px*32px) * (R,G,B)
decoded = tensorflow.keras.layers.Conv2D(3, (3,3), padding='same', name='d9')(decoded)

output = tensorflow.keras.layers.Activation('sigmoid', name='d10')(decoded)


In [ ]:
from tensorflow.keras.models import Model
autoencoder = Model(inputs=input_img, outputs=output)
encoder = Model(inputs=input_img, outputs=encoded)
encoder.summary()

In [ ]:
autoencoder.summary()

In [ ]:
encoded_input = tensorflow.keras.layers.Input((4, 4, 16))
decoded_layer = autoencoder.get_layer('d1')(encoded_input)
decoded_layer = autoencoder.get_layer('d2')(decoded_layer)
decoded_layer = autoencoder.get_layer('d3')(decoded_layer)
decoded_layer = autoencoder.get_layer('d4')(decoded_layer)
decoded_layer = autoencoder.get_layer('d5')(decoded_layer)
decoded_layer = autoencoder.get_layer('d6')(decoded_layer)
decoded_layer = autoencoder.get_layer('d7')(decoded_layer)
decoded_layer = autoencoder.get_layer('d8')(decoded_layer)
decoded_layer = autoencoder.get_layer('d9')(decoded_layer)
decoded_layer = autoencoder.get_layer('d10')(decoded_layer)

decoder = Model(inputs=encoded_input, outputs=decoded_layer)
decoder.summary()

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])

In [ ]:
# Setup saving the model producing the best val_acc
checkpoint_filepath = 'best.h5'
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = 'val_mse',
    mode='min',
    save_best_only=True, verbose=2
)

In [ ]:
autoencoder.fit(X_train, X_train, validation_data=(X_test, X_test), epochs=50, callbacks=[model_checkpoint_callback], batch_size=128)

In [ ]:
autoencoder.load_weights('best.h5')
autoencoder.evaluate(X_test, X_test)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 
decoded_imgs = autoencoder.predict(X_test)
fig, ax = plt.subplots(nrows=2, ncols=10, figsize=(20, 4))
plt.gray()
for indice, row in enumerate(ax):
    for indice2, col in enumerate(row):
        if indice == 0:
            col.imshow(X_test[indice2].reshape(32,32, 3))
        else: 
            col.imshow(decoded_imgs[indice2].reshape(32,32, 3))
plt.show()